In [1]:
import json
import re
import fasttext 
import numpy as np
from tqdm.notebook import tqdm

In [11]:
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
AITopLevelTopics = ['Artificial intelligence', 'Computer vision', 'Data mining',
                     'Data science', 'Machine learning', 'Natural language processing',
                     'Pattern recognition', 'Speech recognition']
with open("./data/dblpPaperIDs2Thresholded.json", 'r') as f:
    paperList = json.load(f)
papersUnderConsideration = set(paperList)

In [3]:
fasttextModel = fasttext.load_model('crawl-300d-2M-subword.bin')

In [5]:
import string
translator = str.maketrans('', '', string.punctuation) 

In [8]:
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    with open('./data/dblpAbstract_2Thresholded_FT_Embeddings.json', 'w') as outfile:
        for line in tqdm(file):
            data = json.loads(line)
            paperID = data['id'] 
            abstractWordList = data.get('abstract',[])
            abstractString = ' '.join(word for word in abstractWordList)
            abstractString = abstractString.replace('\n', ' ').replace('\r', '')
            embedding = fasttextModel.get_sentence_vector(abstractString).tolist()    # while reading use np.asarray to convert to np array
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding
            json.dump(outDict, outfile)
            outfile.write('\n')

In [9]:
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    with open('./data/dblpTitle_2Thresholded_FT_Embeddings.json', 'w') as outfile:
        for line in tqdm(file):
            data = json.loads(line)
            paperID = data['id'] 
            title = data.get('title','')
            title = title.replace('\n', ' ').replace('\r', '')
            embedding = fasttextModel.get_sentence_vector(title).tolist()    # while reading use np.asarray to convert to np array
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding
            json.dump(outDict, outfile)
            outfile.write('\n')

In [8]:
tf.enable_eager_execution()

In [13]:
module_url = "./module/UnivTrans" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
def embed(inputText):
    return model(inputText)

In [29]:
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    with open('./data/dblp_Abstract_2Thresholded_USE_Trans_Embeddings.json', 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data['id'] 
            abstractWordList = data.get('abstract',[])
            abstractString = ' '.join(word for word in abstractWordList)
            abstractString = abstractString.replace('\n', ' ').replace('\r', '')
            embedding = embed([abstractString])[0].numpy()
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding.tolist()
            json.dump(outDict, outfile)
            outfile.write('\n')

INFO:root:count: 0
INFO:root:count: 10000
INFO:root:count: 20000
INFO:root:count: 30000
INFO:root:count: 40000
INFO:root:count: 50000
INFO:root:count: 60000
INFO:root:count: 70000
INFO:root:count: 80000
INFO:root:count: 90000
INFO:root:count: 100000
INFO:root:count: 110000
INFO:root:count: 120000
INFO:root:count: 130000
INFO:root:count: 140000
INFO:root:count: 150000
INFO:root:count: 160000
INFO:root:count: 170000
INFO:root:count: 180000
INFO:root:count: 190000
INFO:root:count: 200000
INFO:root:count: 210000
INFO:root:count: 220000
INFO:root:count: 230000
INFO:root:count: 240000
INFO:root:count: 250000
INFO:root:count: 260000
INFO:root:count: 270000
INFO:root:count: 280000
INFO:root:count: 290000
INFO:root:count: 300000
INFO:root:count: 310000
INFO:root:count: 320000
INFO:root:count: 330000
INFO:root:count: 340000
INFO:root:count: 350000
INFO:root:count: 360000
INFO:root:count: 370000
INFO:root:count: 380000
INFO:root:count: 390000
INFO:root:count: 400000
INFO:root:count: 410000
INFO:r

In [15]:
records = []
paperIDs = []
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    for line in tqdm(file):
            data = json.loads(line)
            paperID = data['id'] 
            title = data.get('title','')
            title = title.replace('\n', ' ').replace('\r', '')
            records.append(title)
            paperIDs.append(paperID)
assert len(records) == len(paperIDs)

In [16]:
n = 100     # block size
records = [records[i:i + n] for i in range(0, len(records), n)]
paperIDs = [paperIDs[i:i + n] for i in range(0, len(paperIDs), n)]

In [34]:
count = 0
with open('./data/dblp_Title_2Thresholded_USE_Trans_Embeddings.json', 'w') as outfile:
    for i in tqdm(range(len(records))):
#         recordSubList = records[i]
        paperIDSubList = paperIDs[i]
        embeddings = embed(records[i]).numpy().tolist()
        for embedding, paperID in zip(embeddings, paperIDSubList):
        
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding#.tolist()
            json.dump(outDict, outfile)
            outfile.write('\n')



In [19]:
lis =[ [1,2,3], [4, 5,6]]
lis2 = [['a','b'], ['c', 'd', 'e']]
list(zip(lis, lis2))

[([1, 2, 3], ['a', 'b']), ([4, 5, 6], ['c', 'd', 'e'])]

In [33]:
for i in range(3):
    embed(records[i]).numpy()

In [27]:
len(records) == len(paperIDs)

True